# Smoothing GPS track

<br/>
<div class="alert alert-block alert-warning" style="padding:1em">
<b>Smoothing:</b> this operation consists to modify observation preserving timestamps and without deleting or creating new observations. Smoothing methods are usually local methods and working with a neighborhood (e.g. kernel).</div>

In **tracklib**, smoothing approaches available are:

* des filtres séquentiels (ou filtres à noyaux), qui travaillent directement dans le domaine spatial ou temporel et avec un noyau local
* des filtres dynamiques (qui nécessitent des informations sur la dynamique du mobile) : Kalman et Markov
* des filtres statistiques (qui étudient par "apprentissage" la distribution des données sur un jeu de données d'exemple. Ici, il y aura Karhunen-Loève (ACP fonctionnelle) qui est au statut de "TO DO"
* des filtres fréquentiels, qui travaillent dans le domaine spectrale pour modifier la trace. Pour l'instant il y a les passe-bas et passe-haut de Fourier et il y aura les ondelette en "TO DO".
